<b><img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250, style="padding: 10px"> 
<p><p><p><p><p><p>
<b>Interactive Image Visualization</b> <br>
Last verified to run on 2022-05-31 with LSST Science Pipelines release w_2022_22<br>
Contact authors: Leanne Guy <br>
Target audience: All DP0 delegates. <br>
Minimum Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

**Credit:** This tutorial was inspired by a notebook originally developed by Keith Bechtol in the context of the LSST Stack Club. It has been updated and extended for DP0.1 by Leanne Guy. Please consider acknowledging Leanne Guy and Keith Bechtol in any publications or software releases that make use of this notebook's contents.

### Learning Objectives

This tutorial, together with tutorial `08b_Interactive_Catalog_Visualization`, introduces three open-source Python libraries that enable powerful interactive visualization of images and catalogs. 
 1. [**HoloViews**](http://holoviews.org): Produce high-quality interactive visualizations easily by annotating plots and images rather than using direct calls to a plotting library
 2. [**Bokeh**](https://bokeh.org): A powerful data visualization library that provides interactive tools including brushing and linking between multiple plots. `Holoviews` + `Bokeh`
 3. [**Datashader**](https://datashader.org): Accurately render very large datasets quickly and flexibly.
  
These packages are part of the [Holoviz](https://holoviz.org/) ecosystem of tools intended for visualization in a web browser and can be used to create quite sophisticated dashboard-like interactive displays and widgets. The goal of this tutorial is to provide an introduction and starting point from which to create more advanced, custom interactive visualizations. Holoviz has a [vibrant and active community](https://discourse.holoviz.org/) where you can ask questions and discuss vizualizations with a global community. 

### Logistics
This notebook is intended to be runnable on `data.lsst.cloud`. Note that occasionally the notebook may seem to stall, or the interactive features may seem disabled. If this happens, usually a restart of the kernel fixes the issue. You might also need to log out of the RSP and start a "large" instance of the JupyterLab environment. In some examples shown in this notebook, the order in which the cells are run is important for understanding the interactive features, so you may want to re-run the set of cells in a given section if you encounter unexpected behavior. Note that some of the examples require manual selection of points on a graph to run correctly.

### Setup

In [ ]:
# General python imports
import numpy as np
import warnings

# Astropy
from astropy.visualization import  ZScaleInterval, AsinhStretch

# LSST imports
from lsst.daf.butler import Butler

# Bokeh and Holoviews for visualization
import bokeh
from bokeh.io import output_notebook
from bokeh.models import HoverTool

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

If a message about \"Patching auth into notebook.base.handlers ...\" appeared above, it is ok to ignore, as are messages about the number of threads.

In [ ]:
# What versions of bokeh and holoviews nd datashader are we working with?
# This is important when referring to online documentation as
# APIs can change between versions.
print("Bokeh version: " + bokeh.__version__)
print("Holoviews version: " + hv.__version__)

In [ ]:
# Ignore warnings
import warnings
warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
# What version of the LSST Science Pipelnes are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

### 1. Data preparation

The basis for any data visualization is the underlying data. In this tutorial we will work with images.  For DP0.1, images can only be accessed via the `Butler` (<a href="https://pipelines.lsst.io/modules/lsst.daf.butler/index.html">documentation</a>), an LSST Science Pipelines software package that allows you to fetch the LSST data you want without having to know its location or format. For more details about using the Butler, please refer to tutorial `04_Intro_to_Butler`. For visualization examples with tabular data, see `08b_Interactive_Catalog_Visualization`. 

We will start by using the butler to retrieve a calexp image (specifying a visit, detector, and band) and a deep coadd image (specifying a tract, patch, and band).

In [ ]:
# Instantiate the Butler initializing it with the repository name and the DP0.1 collection identifier
from lsst.daf.butler import Butler
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

It is ok to ignore a pink FutureWarning from the next cell.

In [ ]:
# Define a calibrated exposure and retrieve it via the Butler
calexpId = {'visit': 192350, 'detector': 175, 'band': 'i'}
calexp = butler.get('calexp', calexpId)
assert calexp is not None
print(f"Visit: {calexp.info.getId()}, Band:{calexp.getFilterLabel().bandLabel}, Detector: {calexp.detector.getId()}")

# Note: This will trigger a warning from CFITSIO in w_2022_22. This warning can be safely ignored and will be corrected in the future.

In [ ]:
# Source table for this calexp exposure
calexpSrc = butler.get('src', **calexpId)

In [ ]:
# Define a deep coadded image and retrieve it via the Butler
coaddId = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd = butler.get('deepCoadd', coaddId)
assert coadd is not None

Let's get some infomation about the coadd, such as which visits went into constructing it. As an exercise, explore the information in the coaddInfo object. 

In [ ]:
coaddInfo = coadd.getInfo()

# Which visits went into constructing this coadd?
coaddVisits = coaddInfo.getCoaddInputs().visits
coaddVisits.asAstropy()

In [ ]:
# Source table for this coadd
coaddSrc = butler.get('deepCoadd_forced_src', coaddId)

### 2. Holoviews

[Holoviews](https://holoviews.org) supports easy analysis and visualization by annotating data rather than utilizing direct calls to plotting packages. For this tutorial, we will use [Bokeh](hrrps://bokeh.org) as the plotting library backend for Holoviews. This is defined in the `Setup` section above with the `hv.extension('bokeh')` call.  Holoviews supports several plotting libraries and there is an exercise to the user at the end of this section to explore using Holoviews with other plotting packages. 

The basic core primitives of Holoviews are [Elements](http://holoviews.org/Reference_Manual/holoviews.element.html); hv.Element. Elements are simple wrappers around your data that provide a semantically meaningful visual representation. An Element may be a set of Points, an Image, a Curve, a Histogram, a Scatter, etc. See the Holoviews [Reference Gallery](http://holoviews.org/reference/index.html) for all the various types of Elements that can be created with Holoviews. 


#### 2.2. Visualizing exposure images with Holoviews

In this first example we will use the Holoviews [Image Element](http://holoviews.org/reference/elements/bokeh/Image.html) to quickly visualize the catalog data retrieved in section 1 as a scatter plot. HoloViews maintains a strict separation between content and presentation. This separation is achieved by maintaining sets of keyword values as `options` that specify how `Elements` are to appear.  In this first example we will apply the default options and remove the toolbar. 

In the tutorial `03_Image_Display_and_Manipulation` we saw how to use the `lsst.afw.display` library to visualize exposure images and in tutorial `03b_Image_Display_with_Firefly` we saw how to do the same using Firefly. In this example we demonstrate image visualization at the pixel level with Holoviews.

We will use the holoviews Image Element to visualize a calexp. We will then overlay a Holoviews DynamicMap on the image to compute and display elements dynamically, allowing exploration of large datasets. DynamicMaps generate elements on the fly allowing exploration of parameters with arbitrary resolution. DynamicMaps are lazy in the sense they only compute as much data as the user wishes to explore. An Overlay is a collection of HoloViews objects that are displayed simultanously, e.g a Curve superimposed on a Scatter plot of data. You can build a Overlay between any two HoloViews objects, which can have different types using the * operator. 

First, we will use the `astropy.visualization` library to define an asinh stretch and zscale interval and apply them to the calexp object. These are the same transformations that were applied in `03_Image_Display_and_Manipulation` notebook.

In [ ]:
# Apply a asinh/zscale mapping to the data 
transform = AsinhStretch() + ZScaleInterval()
scaledImage = transform(calexp.image.array)

LSST’s image classes (Image, Mask, MaskedImage, and Exposure) use a pixel indexing convention that is different from both the convention used by `numpy.ndarray` objects and the convention used in FITS images (as documented [here](https://pipelines.lsst.io/modules/lsst.afw.image/indexing-conventions.html)).  Most plotting tools assume pixel (0, 0) is in the upper left where we always assume (0,0) is in the lower left. Consequently, we flip the data array. 

In [ ]:
scaledImage = np.flipud(scaledImage)
bounds_img = (0, 0, calexp.getDimensions()[0], calexp.getDimensions()[1])

Further details can be found at [Image Indexing, Array Views, and Bounding Boxes](https://pipelines.lsst.io/modules/lsst.afw.image/indexing-conventions.html) in the Rubin Science Pipelines and Data Products. 

In [ ]:
# Define some default plot options for the Image
img_opts = dict(height=600, width=700, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '8pt'},
                colorbar=True, toolbar='right', show_grid=True,
                tools=['hover']
               )     

In [ ]:
# Make a function to autogenerate a plot title from the dataId.
def dataIdToString(dataId: dict) -> str:
    title = "DC2 image: "
    for key, value in dataId.items():
        title += str(key) + ": " + str(value) + " "
    return title.strip() 

In [ ]:
# Create the Image element.
img = hv.Image(scaledImage, bounds=bounds_img,
               kdims=['x', 'y']).opts(
    cmap = "Greys_r",  xlabel = 'X', ylabel ='Y',
    title = dataIdToString(calexpId),
    **img_opts)

In [ ]:
rasterize(img)

#### 2.3 Overlaying source detections on an image

Now let's overlay the source detections from the `Source` catalog on this image. We will use the Points Element for the detections to overlay.

In [ ]:
coords = calexpSrc.getX(), calexpSrc.getY()

In [ ]:
f"Number of src detections is, {len(coords[1])}"

In [ ]:
# Custom hover tool for the source detections
detHoverTool = HoverTool(
    tooltips=[
        ( 'X', '@x{0.2f}'),
        ( 'Y', '@y{0.2f}'),
    ],
    formatters={
        'X' : 'printf',
        'Y' : 'printf',
    },
    
)
detections = hv.Points(coords).opts(
    fill_color=None, size = 9, color="darkorange",
    tools=[detHoverTool])

Now we overlay the detected sources on the image. The `*` operator  is used to overlay one Element on to another.  

In [ ]:
# Reset the tools on the image and add a hover on the detections.
rasterize(img).opts(tools=[]) * detections.opts(tools=[detHoverTool])

You can now  mouse-over the sources and get the coordinates of the detections. 

### 3.0 Interactive Image Exploration with with Holoviews Streams and DynamicMap

Now let's add some interactive exploration capability using Holoviews [Streams](http://holoviews.org/user_guide/Streaming_Data.html) and [DynamicMap](https://holoviews.org/reference/containers/bokeh/DynamicMap.html). A DynamicMap is an explorable multi-dimensional wrapper around a callable that returns HoloViews objects. The core concept behind a stream is simple: it defines one or more parameters that can change over time that automatically refreshes code depending on those parameter values.

First create a DynamicMap with a box stream so that we can explore selected sections of the image.

In [ ]:
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=img, bounds=boundsxy)
dynamicMap = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

In [ ]:
# Display the image and overlay the DynamicMap
rasterize(img) * dynamicMap

Using the interactive callback features on the image plots, such as the selection box (the icon of the box with a '+' in the lower right corner), we can explore regions of the image.  Use the box select tool on the image above to select a region and then execute the cell below to get the box boundary coordinates. 

In [ ]:
box

Below is another version of the image with a [tap stream](http://holoviews.org/reference/streams/bokeh/Tap.html) instead of box select. A Tap stream allows you to click or 'tap' a position to interact with a plot. Try zooming in on an interesting part of the image and then 'tap' somewhere to place an 'X' marker. 

In [ ]:
posxy = hv.streams.Tap(source=img, x=0.5 * calexp.getDimensions()[0],
                       y=0.5 * calexp.getDimensions()[1])
marker = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[posxy])
rasterize(img)* marker.opts(color='white', marker='x', size=20)

'X' marks the spot! What's the value at that location? Execute the next cell to find out.

In [ ]:
print('The scaled/raw value at position (%.3f, %.3f) is %.3f / %.3f' %
      (posxy.x, posxy.y, scaledImage[-int(posxy.y), int(posxy.x)], 
       calexp.image.array[-int(posxy.y), int(posxy.x)]))

### 4.0  Optional exercises to the user 

 1. Holoviews works with a wide range of plotting libraries, Bokeh, matplotlib, plotly, mpld3, pygal to name a few. As an exercise, try changing the Holoviews plotting library to be `matplotlib` instead of `bokeh` in the `Setup` cell at the beginning of the notebook with `hv.extension('matplotlib')`. You will see the holoviews + matplotlib icons displayed when the library is loaded successfully. Run the cells in section 2.1 again and compare the outputs. Try again with some other plotting library. Don't forget to set the plotting library back to whichever you prefer to use for the rest of this tutorial.
 2. In the image display sections, try using the coadd image instead of the calexp image. 
 
 3. In section 2.3, try extracting additional information about the detected sources from the calexpSrc table and adding it to the custom hover tool. For example, the corresponding RA/DEC or the PSF flux. 
 
 3. Try using a different stream function to interact with the images in section 3  